In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame (
    [[8,8,4],[7,9,5],[6,10,6],[5,12,7],[4,10,5]],
    columns=['internships', 'profile_score', 'lpa']
)

df

,internships,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,7
4,4,10,5


In [3]:
# layers will contain the size of each layer
def initialize(layers): # initialize all the weights to 1 and bias to 0
    parameters = {}

    for l in range (1, len(layers)):
        parameters['W'+str(l)] = np.ones((layers[l-1], layers[l]))
        parameters['b'+str(l)] = np.zeros((layers[l], 1))

    return parameters


In [4]:
def layer_calc(A_prev, w, b):
    Z = np.dot(w.T, A_prev) + b
    return Z

In [5]:
def forward_propagation(X, parameters):
    A = X
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(1, L+1):
        A_prev = A
        wl = parameters['W'+str(l)]
        bl = parameters['b'+str(l)]

        A = layer_calc(A_prev, wl, bl)

    y_hat = A[0][0]
    return y_hat, A_prev

In [6]:
def back_propogation(parameters, y, y_hat, A1, X): #updating the parameters
    learning_rate = 0.001
    parameters['W2'][0][0] = parameters['W2'][0][0] + (learning_rate * 2 * (y - y_hat)*A1[0][0])
    parameters['W2'][1][0] = parameters['W2'][1][0] + (learning_rate * 2 * (y - y_hat)*A1[1][0])
    parameters['b2'][0][0] = parameters['W2'][1][0] + (learning_rate * 2 * (y - y_hat))

    parameters['W1'][0][0] = parameters['W1'][0][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0]*X[0][0])
    parameters['W1'][0][1] = parameters['W1'][0][1] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0]*X[1][0])
    parameters['b1'][0][0] = parameters['b1'][0][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0])

    parameters['W1'][1][0] = parameters['W1'][1][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0]*X[0][0])
    parameters['W1'][1][1] = parameters['W1'][1][1] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0]*X[1][0])
    parameters['b1'][1][0] = parameters['b1'][1][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0])

In [7]:
parameters = initialize([2, 2, 1]) #network with 3 layers
epochs = 100

for i in range(1, epochs+1):
    loss = []

    for j in range(df.shape[0]): #iterating over all rows in the df
        X = df[['internships', 'profile_score']].values[j].reshape(2,1)
        y = df[['lpa']].values[j][0]

        y_hat, A1 = forward_propagation(X, parameters)
        back_propogation(parameters, y, y_hat, A1, X)

        loss.append((y-y_hat)**2) # Loss function is MSE and Activation Function is Linear

    if(i%5==0):
        print(f"Epoch - {i}, Loss - {np.array(loss).mean()}")

print("Final Parameters: ", parameters)

Epoch - 5, Loss - 1.3600493243139717
Epoch - 10, Loss - 1.3558815531516404
Epoch - 15, Loss - 1.3514182280379372
Epoch - 20, Loss - 1.3467118577834853
Epoch - 25, Loss - 1.3418193114196404
Epoch - 30, Loss - 1.336801511054055
Epoch - 35, Loss - 1.3317231273566086
Epoch - 40, Loss - 1.3266522919934747
Epoch - 45, Loss - 1.321660340174584
Epoch - 50, Loss - 1.3168215947312998
Epoch - 55, Loss - 1.312213200820438
Epoch - 60, Loss - 1.30791501744501
Epoch - 65, Loss - 1.3040095684175483
Epoch - 70, Loss - 1.3005820510017434
Epoch - 75, Loss - 1.2977203949557246
Epoch - 80, Loss - 1.295515357590462
Epoch - 85, Loss - 1.2940606310338993
Epoch - 90, Loss - 1.293452925119145
Epoch - 95, Loss - 1.2937919717318809
Epoch - 100, Loss - 1.2951803720477006
Final Parameters:  {'W1': array([[1.05133958, 1.19375513],
       [1.06461087, 1.2072599 ]]), 'b1': array([[0.01010137],
       [0.0118249 ]]), 'W2': array([[0.15361836],
       [0.14886973]]), 'b2': array([[0.14685328]])}


# Using Keras

In [8]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input

In [9]:
model = Sequential()

model.add(Input(shape=(2,)))
model.add(Dense(2, activation='linear'))
model.add(Dense(1, activation='linear'))

In [10]:
optimizer = tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.01)
# optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.get_weights()

[array([[-0.60493106,  0.41120386],
        [ 1.1746551 , -0.807788  ]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[-0.39700377],
        [ 1.3876363 ]], dtype=float32),
 array([0.], dtype=float32)]

In [13]:
model.set_weights([
    np.array([[1,1],[1,1]]), np.array([0,0]), np.array([[1],[1]]), np.array([0])
])

In [14]:
model.fit(df[['internships', 'profile_score']].values, df['lpa'], epochs=100, verbose=1, batch_size=1)

Epoch 1/100
5/5 [==============================] - 0s 929us/step - loss: 620.8372
Epoch 2/100
5/5 [==============================] - 0s 594us/step - loss: 479.7785
Epoch 3/100
5/5 [==============================] - 0s 643us/step - loss: 359.9280
Epoch 4/100
5/5 [==============================] - 0s 607us/step - loss: 270.0307
Epoch 5/100
5/5 [==============================] - 0s 589us/step - loss: 198.7092
Epoch 6/100
5/5 [==============================] - 0s 674us/step - loss: 146.4005
Epoch 7/100
5/5 [==============================] - 0s 665us/step - loss: 109.8160
Epoch 8/100
5/5 [==============================] - 0s 630us/step - loss: 80.0367
Epoch 9/100
5/5 [==============================] - 0s 542us/step - loss: 59.2666
Epoch 10/100
5/5 [==============================] - 0s 591us/step - loss: 44.7520
Epoch 11/100
5/5 [==============================] - 0s 655us/step - loss: 33.3604
Epoch 12/100
5/5 [==============================] - 0s 663us/step - loss: 25.7454
Epoch 13/100
5/5 [

In [15]:
model.get_weights()

[array([[0.40770516, 0.40770516],
        [0.48700055, 0.48700055]], dtype=float32),
 array([-0.5553082, -0.5553082], dtype=float32),
 array([[0.45852244],
        [0.45852244]], dtype=float32),
 array([-0.6782013], dtype=float32)]